In [14]:
import numpy as np
import scipy.io as io
from scipy.special import expit
from time import sleep
from sklearn.model_selection import ParameterGrid, KFold
from sklearn.metrics import confusion_matrix, accuracy_score
from sklearn.gaussian_process import GaussianProcessClassifier
from sklearn.gaussian_process.kernels import RBF
from sklearn.svm import SVC
from skrvm import RVC
from tqdm import tqdm

from multiprocessing import Pool

In [15]:
# Remove warning generated by classifier libraries.
import warnings
warnings.filterwarnings('ignore')

In [16]:
def MyConfusionMatrix(Y,ClassNames):
    print(confusion_matrix(ClassNames, Y))
    return confusion_matrix(ClassNames, Y),accuracy_score(ClassNames, Y)

## Helper functions for SVM

In [17]:
'''============================ Predicts probability of labels for RVM ==============================================='''
def predict_proba(clf,XValidate):
    noOfClasses = np.shape(clf.classes_)[0]
    noOfClassifiers = (noOfClasses * (noOfClasses-1))/2
    print('no of Classifiers',noOfClassifiers)
    dataSize = np.shape(XValidate)[0]
    Yvalidate = np.zeros((dataSize, np.shape(clf.classes_)[0]))
    c = 0
    prob = clf.multi_.estimators_[c].predict_proba(XValidate)
    #Summing Fkm(X) where k!=m
    for i in range(0,noOfClasses):
        for j in range(i, noOfClasses):
            if (i < j):
                Yvalidate[:, i] =  Yvalidate[:, i]+ prob[:, 0]
                Yvalidate[:, j] =  Yvalidate[:, j]+ prob[:, 1]
                c = c + 1;
                if(c<noOfClassifiers):
                    prob = clf.multi_.estimators_[c].predict_proba(XValidate)
    #Calculating 1/G(summation(ykm))
    Yvalidate = Yvalidate / np.shape(clf.classes_)[0]
    #Calculating probability of XValidate  not belonging to any class
    prob_std = np.ndarray.std(Yvalidate, axis=1)[:, np.newaxis]
    sigmoid = 1 - expit(prob_std)
    Yvalidate = np.concatenate([Yvalidate, sigmoid], axis=1)
    Yvalidate = Yvalidate / np.repeat((sigmoid + 1), axis=1, repeats=np.shape(clf.classes_)[0] + 1)
    return Yvalidate


In [18]:
'''====================== Method to get the Parameters and the HyperParameters ====================================='''
def get_params(clf):
    if np.shape(clf.classes_)[0] == 2:
        Parameters = [{'phi': clf.phi,'relevance': clf.relevance_,'alpha': clf.alpha_,'beta': clf.beta_,'m': clf.m_,
                       'gamma': clf.gamma,'bias': clf.bias,'clf': clf}]
    else :
        Parameters = [clf]
        for c in clf.multi_.estimators_:
            Parameter = {'phi': c.phi,'relevance': c.relevance_,'alpha': c.alpha_,'beta': c.beta_,'m': c.m_,
                         'gamma': c.gamma,'bias': c.bias,'clf': c}
            Parameters.append(Parameter)
    return Parameters

In [19]:
def get_svc(parameters):
    svc = SVC(C=parameters['parameters']['C'], kernel=parameters['parameters']['kernel'],
              degree=parameters['parameters']['degree'], gamma=parameters['parameters']['gamma'],
              coef0=parameters['parameters']['coef0'], probability=parameters['parameters']['probability'],
              tol=parameters['parameters']['tol'], cache_size=parameters['parameters']['cache_size'],
              class_weight=parameters['parameters']['class_weight'], shrinking=parameters['parameters']['shrinking'],
              verbose=parameters['parameters']['verbose'], max_iter=parameters['parameters']['max_iter'],
              decision_function_shape=parameters['parameters']['decision_function_shape'],
              random_state=parameters['parameters']['random_state'])
    return svc


def svc_train(svc, estimate, labels):
    svc.fit(estimate, labels)


def svc_predict(svc, validate):
    return svc.predict(validate)


def svc_probability(svc, validate):
    return svc.predict_proba(validate)


def svc_score(svc, validate, v_labels):
    return svc.score(validate, v_labels)


def svc_get_para(svc):
    support = svc.support_
    support_vectors = svc.support_vectors_
    n_support = svc.n_support_
    dual_coef = svc.dual_coef_
    intercept = svc.intercept_
    sparse = svc._sparse
    shape_fit = svc.shape_fit_
    prob_a = svc.probA_
    prob_b = svc.probB_
    gamma = svc._gamma
    classes = svc.classes_
    hyper = svc.get_params(deep=True)
    ret = {'support': support, 'support_vectors': support_vectors, 'n_support': n_support, 'dual_coef': dual_coef,
           'intercept': intercept, 'sparse': sparse, 'shape_fit': shape_fit, 'prob_a': prob_a, 'prob_b': prob_b,
           'gamma': gamma, 'classes': classes, 'hyper': hyper}
    return ret


def svc_set_para(svc, svc_para):
    svc.set_params(**svc_para['hyper'])
    svc.support_ = svc_para['support']
    svc.support_vectors_ = svc_para['support_vectors']
    svc.n_support_ = svc_para['n_support']
    svc._dual_coef_ = svc_para['dual_coef']
    svc._intercept_ = svc_para['intercept']
    svc._sparse = svc_para['sparse']
    svc.shape_fit_ = svc_para['shape_fit']
    svc.probA_ = svc_para['prob_a']
    svc.probB_ = svc_para['prob_b']
    svc._gamma = svc_para['gamma']
    svc.classes_ = svc_para['classes']

## Helper funtions for MyCrossValidate 

In [20]:
def CVhelper(args):
    XEstimate=args[0]
    XValidate= args[1]
    YEstimate=args[2]
    YValidateTrue=args[3]
    algo=args[4]
    params=args[5]
                
    # calling TrainMyClassifier
    YValidate, Estparameters = TrainMyClassifier(XEstimate, YEstimate, XValidate,
                                                 {'algorithm':algo, 'parameters':params })
                
                
    score = (YValidateTrue == np.sum(np.argmax(YValidate, axis=1))/np.size(YValidateTrue))*100
    return score

In [21]:
def TrainMyClassifier(XEstimate, ClassLabels, XValidate, Parameters ):
    if Parameters['algorithm'] == 'RVM':
        Parameters = Parameters['parameters']
        clf =  RVC(alpha=Parameters.get('alpha'),beta=Parameters.get('beta'),n_iter=Parameters.get('n_iter'))
        clf.fit(XEstimate, ClassLabels)
        if np.shape(clf.classes_)[0] == 2:
            Yvalidate = clf.predict_proba(XValidate)
        else:
            Yvalidate = predict_proba(clf,XValidate)
        EstParameters = get_params(clf)
        return Yvalidate,EstParameters

    elif Parameters['algorithm'] == 'SVM':
        
        svc = get_svc(Parameters)
        svc_train(svc, XEstimate, ClassLabels)
        prob = svc_probability(svc, XValidate)
        EstParameters = svc_get_para(svc)
        
        prob_std = np.ndarray.std(prob, axis=1)[:, np.newaxis]
        sigmoid = 1 - expit(prob_std)
        Yvalidate = np.concatenate([prob, sigmoid], axis=1)
        Yvalidate = Yvalidate / np.repeat((sigmoid + 1), axis=1, repeats=len(svc.classes_)+1)
        
        return Yvalidate, EstParameters

    elif Parameters["algorithm"] == "GPR":
        # get the classes from the labels
        classes = np.unique(ClassLabels, axis=0)
        sorted(classes, reverse=True)
        num_class = len(classes)

        # get data and label based on classes
        data = []
        for cla in classes:
            data.append(XEstimate[ClassLabels == cla])

        target = []
        for cla in classes:
            target.append(ClassLabels[ClassLabels == cla])

        # put data and label into a matrix, so that we could do a easier calculation for probability
        # the following calculation is all based on the matrix
        data_matrix = []
        for i in range(num_class - 1):
            data_matrix.append([])
            for j in range(num_class - 1):
                data_matrix[i].append(None)

        target_matrix = []
        for i in range(num_class - 1):
            target_matrix.append([])
            for j in range(num_class - 1):
                target_matrix[i].append(None)

        for i in range(num_class-1):
            for j in range(i, num_class-1):
                data_matrix[i][j] = np.concatenate([data[i], data[j+1]], axis=0)
                target_matrix[i][j] = np.concatenate([target[i], target[j+1]], axis=0)

        classifier_matrix = []
        for i in range(num_class-1):
            classifier_matrix.append([])
            for j in range(num_class-1):
                classifier_matrix[i].append(None)

        for i in range(num_class-1):
            for j in range(i, num_class-1):
                gpc_classifier = GaussianProcessClassifier(
                    kernel=Parameters["parameters"]["kernel"],
                    optimizer=Parameters["parameters"]["optimizer"],
                    n_restarts_optimizer=Parameters["parameters"]["n_restarts_optimizer"],
                    max_iter_predict=Parameters["parameters"]["max_iter_predict"],
                    warm_start=Parameters["parameters"]["warm_start"],
                    copy_X_train=Parameters["parameters"]["copy_X_train"],
                    random_state=Parameters["parameters"]["random_state"],
                    multi_class="one_vs_rest",
                    n_jobs=Parameters["parameters"]["n_jobs"]
                )
                gpc_classifier.fit(data_matrix[i][j], target_matrix[i][j])
                classifier_matrix[i][j] = gpc_classifier

        out_matrix = []
        for i in range(num_class-1):
            out_matrix.append([])
            for j in range(num_class-1):
                out_matrix[i].append(None)

        for i in range(num_class-1):
            for j in range(i, num_class-1):
                out_matrix[i][j] = classifier_matrix[i][j].predict_proba(XValidate)

        # calculate the whole prediction prob
        val_shape = XValidate.shape[0]
        predict_prob_list = []
        for i in range(num_class):
            predict_prob_list.append(np.zeros(shape=[val_shape, 1]))

        for i in range(num_class-1):
            for j in range(i, num_class-1):
                predict_prob_list[i] += out_matrix[i][j][:, 0][:, np.newaxis] / (num_class * 2)
                predict_prob_list[j + 1] += out_matrix[i][j][:, 1][:, np.newaxis] / (num_class * 2)

        # get the result of num_class probability
        result = np.concatenate(predict_prob_list, axis=1)

        # calculate the probability for the one more class
        std = np.std(result, axis=1)[:, np.newaxis]
        other_prob = np.exp(-std) / (1 + np.exp(std * 5))
        result = np.concatenate([result, other_prob], axis=1)
        result = result / np.repeat((other_prob + 1), axis=1, repeats=num_class + 1)

        # put all the parameters into a dict
        estParameters = {}
        estParameters["class_num"] = num_class
        estParameters["parameters"] = []
        for i in range(num_class-1):
            for j in range(i, num_class-1):
                estParameters["parameters"].append(
                    {
                        "log_marginal_likelihood_value_": classifier_matrix[i][j].log_marginal_likelihood_value_,
                        "classes_": classifier_matrix[i][j].classes_,
                        "n_classes_": classifier_matrix[i][j].n_classes_,
                        "base_estimator_": classifier_matrix[i][j].base_estimator_
                    }
                )

        return result, estParameters    

In [22]:
def MyCrossValidate(XTrain, ClassLabels, Nf):
    """
    Xtrain: Training data with labels
    ClassLabels: Class labels for train set.
    Nf: Number of folds
    
    returns:
            Array of Ytrain:
            Array of EstParameters:
            Array of EstConfMatrices:
            Array of ConfMatrix:
    """
    
    algorithms = ['RVM']
    parameters = {'SVM':{'C' : [1, 5, 10], 'kernel' : ['rbf','poly'], 'degree' : [2, 3, 5], 'gamma' : ['auto'],
                            'coef0' : [0.0], 'probability' : [True], 'shrinking' : [True], 'tol' : [1e-3, 1e-4],
                            'class_weight' : ['balanced'], 'verbose' : [False], 'max_iter' : [-1],
                            'decision_function_shape' : ['ovo'], 'random_state' : [None], 'cache_size': [800]},
                  
                 'RVM':{ 'kernel':['rbf'],'degree':[3],'coef1':[None],'coef0':[0.0],'n_iter':[500],'tol':[1e-3],
                        'alpha':[100,1,1e-6],'threshold_alpha':[1e9],'beta':[1.e-2 ,1.e-6, 1.e-8],'beta_fixed':[False],
                        'bias_used':[True],'verbose':[False]},
                  
                 'GPR':{"kernel": [ 1.0 * RBF(1), 
                                    2.0 * RBF(1),
                                    3.0 * RBF(1),
                                    1.0 * RBF(2),
                                    1.0 * RBF(3)
                                    ],
                              "optimizer": ["fmin_l_bfgs_b"],
                              "n_restarts_optimizer": [0],
                              "max_iter_predict": [100],
                              "warm_start": [True],
                              "copy_X_train": [True],
                              "random_state": [0],
                              "multi_class": ["one_vs_one"],
                              "n_jobs": [-1]}}
    
    # de code ClassLabels to numbers 
    ClassLabels = np.argmax(ClassLabels, axis=1)
    
    
    algo_score = []
    algo_params = []
    
    Ytrain = []
    EstParameter = []
    EstConfMatrices = []
    ConfMatrix = []
    for algo in algorithms:
        
        # generating hyper parameter array for hyper parameter search.
        grid = ParameterGrid(parameters[algo])
        grid_search_score = []
        pbar = tqdm(list(grid))
        for params in pbar:
            pbar.set_description("Searching Parameters for {}".format(algo))
            # scikit-learn object to divide data set in Estimate and Validate sets.
            k_fold = KFold(n_splits=Nf, random_state=None, shuffle=False)
            
            # Array of scores for each split.
            cv_scores = []
            p = Pool(5)
            p_params =[]
            for train_index, val_index in k_fold.split(XTrain):

                # Spliting data in Estimate and validate set.
                XEstimate, XValidate = XTrain[train_index], XTrain[val_index]
                YEstimate, YValidateTrue = ClassLabels[train_index], ClassLabels[val_index]
                p_params.append([XEstimate, XValidate, YEstimate, YValidateTrue, algo, params])
            
            cv_scores = p.map(CVhelper, p_params)
            
            # average accuracy for selected hyper parameters 
            score = np.mean(cv_scores)
            grid_search_score.append(score)

        # calculating best hyper parameters
        idx = np.argmax(grid_search_score)
        best_params = list(grid)[idx]
        
        # storing best algorithm score and hyper parameters
        algo_score.append(np.max(grid_search_score))
        algo_params.append(best_params)
        
        
        # calculating method out for current algorithm
        _Ytrain = []
        _EstParameter = []
        _EstConfMatrices = []
        _ConfMatrix = []

        # scikit-learn object to divide data set in Estimate and Validate sets.
        k_fold = KFold(n_splits=Nf, random_state=None, shuffle=False)

        _all_Yval_true = []
        _all_Yval_pred = []
        index = 0
        
        print("Average cross validation score for {} is {}%.".format(algo, np.max(grid_search_score)))
        for train_index, val_index in k_fold.split(XTrain):

            # Spliting data in Estimate and validate set.
            XEstimate, XValidate = XTrain[train_index], XTrain[val_index]
            YEstimate, YValidateTrue = ClassLabels[train_index], ClassLabels[val_index]

            # calling TrainMyClassifier
            YValidate, Estparameters = TrainMyClassifier(XEstimate, YEstimate, XValidate,
                                                         {'algorithm':algo, 'parameters':best_params })
            
            # storing TrainMyClassifier's output  
            _Ytrain.append(YValidate)
            _EstParameter.append(Estparameters)
            
            # printing kernel numbers 
            if  algo == 'SVM':
                print('Number of support vector for validation set {}: {}'.format(index, len(Estparameters['support_vectors'])))
            elif  algo == 'RVM':
                print('Number of relavance vector for validation set {}: {}'.format(index, len(Estparameters['relevant_vectors'])))
                
            # calculating confusion matrix for validation set
            print("Confusion matrix for validation set {}: ".format(index))
            _EstConfMatrices.append(MyConfusionMatrix(np.argmax(YValidate, axis=1), YValidateTrue))
            
            
            _all_Yval_true.extend(YValidateTrue.tolist())
            _all_Yval_pred.extend(np.argmax(YValidate, axis=1).tolist())
            
            index+=1
        print("Over all Confusion matrix for {}: ".format(algo))
        # calculating over all confusion matrix for validation set
        _all_Yval_pred = np.array(_all_Yval_pred).reshape(len(_all_Yval_pred),1)
        _all_Yval_true = np.array(_all_Yval_true).reshape(len(_all_Yval_true),1)
        _ConfMatrix = MyConfusionMatrix(_all_Yval_pred, _all_Yval_true)
        
        # append Main outputs
        Ytrain.append(_Ytrain)
        EstParameter.append(_EstParameter)
        EstConfMatrices.append(_EstConfMatrices)
        ConfMatrix.append(_ConfMatrix)
        
        # waiting for all print commands to execute.
        sleep(.5)
        
    return Ytrain, EstParameter, EstConfMatrices, ConfMatrix

In [23]:
train = io.loadmat("/Users/devanshusingh/PycharmProjects/ML/ML2/Proj2FeatVecsSet1.mat")['Proj2FeatVecsSet1']
# label shape is [25000, 5] as [num_sample, num_class]
label = io.loadmat("/Users/devanshusingh/PycharmProjects/ML/ML2/Proj2TargetOutputsSet1.mat")['Proj2TargetOutputsSet1']
data = np.concatenate((train, label), axis=1)
np.random.shuffle(data)
train = data[:,:-5]
label = data[:,-5:]

In [ ]:
Ytrain, EstParameter, EstConfMatrices, ConfMatrix = MyCrossValidate(train, label, 5)

Searching Parameters for RVM:   0%|          | 0/9 [00:00<?, ?it/s]

In [ ]:
np.save("Ytrain", Ytrain)
np.save("EstParameter", EstParameter)
np.save("EstConfMatrices",EstConfMatrices )
np.save("ConfMatrix", ConfMatrix)

In [24]:
def TestMyClassifier(XTest,Parameters,EstParameters):
    if Parameters['algorithm'] == 'SVM':
        n_svc = get_svc(Parameters)
        svc_set_para(n_svc, EstParameters)
        Ytest = svc_probability(n_svc, XTest)
        
        return Ytest
    
    elif Parameters['algorithm'] == 'RVM':
        if len(EstParameters) == 1:
            clf = EstParameters.get('clf')
        else:
            clf = EstParameters[0]
        if np.shape(clf.classes_)[0] == 2:
            Ytest = clf.predict_proba(XTest)
        else:
            Ytest = predict_proba(clf,XTest)
            
        return Ytest

    elif Parameters['algorithm'] == 'GPR':
        num_class = EstParameters["class_num"]
        classifier = []
        # init all the classifiers
        for param_dict in EstParameters["parameters"]:
            gpc_classifier = GaussianProcessClassifier(
                kernel=Parameters["parameters"]["kernel"],
                optimizer=Parameters["parameters"]["optimizer"],
                n_restarts_optimizer=Parameters["parameters"]["n_restarts_optimizer"],
                max_iter_predict=Parameters["parameters"]["max_iter_predict"],
                warm_start=Parameters["parameters"]["warm_start"],
                copy_X_train=Parameters["parameters"]["copy_X_train"],
                random_state=Parameters["parameters"]["random_state"],
                multi_class="one_vs_rest",
                n_jobs=Parameters["parameters"]["n_jobs"]
            )
            gpc_classifier.log_marginal_likelihood_value_ = param_dict["log_marginal_likelihood_value_"]
            gpc_classifier.classes_ = param_dict["classes_"]
            gpc_classifier.n_classes_ = param_dict["n_classes_"]
            gpc_classifier.base_estimator_ = param_dict["base_estimator_"]
            classifier.append(gpc_classifier)

        # put all the classifiers into a matrix, so it is easier for calculation
        classifier_matrix = []
        for i in range(num_class-1):
            classifier_matrix.append([])
            for j in range(num_class-1):
                classifier_matrix[i].append(None)

        count = 0
        for i in range(num_class-1):
            for j in range(i, num_class-1):
                classifier_matrix[i][j] = classifier[count]
                count += 1

        # calculate the output for XTest
        out_matrix = []
        for i in range(num_class - 1):
            out_matrix.append([])
            for j in range(num_class - 1):
                out_matrix[i].append(None)

        for i in range(num_class - 1):
            for j in range(i, num_class - 1):
                out_matrix[i][j] = classifier_matrix[i][j].predict_proba(XTest)

        # calculate the whole prediction prob
        val_shape = XTest.shape[0]
        predict_prob_list = []
        for i in range(num_class):
            predict_prob_list.append(np.zeros(shape=[val_shape, 1]))

        for i in range(num_class - 1):
            for j in range(i, num_class - 1):
                predict_prob_list[i] += out_matrix[i][j][:, 0][:, np.newaxis] / (num_class * 2)
                predict_prob_list[j + 1] += out_matrix[i][j][:, 1][:, np.newaxis] / (num_class * 2)

        result = np.concatenate(predict_prob_list, axis=1)

        # calculate the probability for the one more class
        std = np.std(result, axis=1)[:, np.newaxis]
        other_prob = np.exp(-std) / (1 + np.exp(std * 5))
        result = np.concatenate([result, other_prob], axis=1)
        result = result / np.repeat((other_prob + 1), axis=1, repeats=num_class + 1)

        return result
